# Practice Text classification with Naive Bayes  
        
        
        
<h3>Abstract</h3>
<p>We will do text classification on a collection of Dutch parliamentary questions.
    The website <a href="https://zoek.officielebekendmakingen.nl/zoeken/parlementaire_documenten">officielebekendmakingen.nl</a>lets you search in "kamervragen".
    <!--You can donwload
    <a href='http://data.politicalmashup.nl/kamervragen/PoliDocs_Kamervragen.zip'>this zipfile with Kamervragen in XML</a>
    to see some of the  data in XML format. 
    It also contains style sheets to show the XML well in a browser.  
-->
    The <a href='http://maartenmarx.nl/teaching/zoekmachines/LectureNotes/MySQL/'>MYSQL directory</a> contains an <a href='http://maartenmarx.nl/teaching/zoekmachines/LectureNotes/MySQL/KVR14807.xml'>example   Kamervraag XML file</a> and a file `kvr.csv.gz` with 40K kamervragen in a handy csv format. Note that in your browser you see the result of applying stylesheets. So choose View Source or open it in an editor.</p>

<h3>First exploration</h3>

See below.

<h2>Exercises</h2>

<p>We will use the fields in elements of the form <tt> &lt;item attribuut="Afkomstig_van"></tt> as our classes. 
    These are the ministeries to whom the question is addressed.
    An example is 
    <pre>
        &lt;item attribuut="Afkomstig_van">Landbouw, Natuurbeheer en Visserij (LNV)&lt;/item>
    </pre>
    Note that these labels are <strong>not normalized</strong>, see e.g. the counts below:
    <pre>
Justitie (JUS)                                                   3219
Volksgezondheid, Welzijn en Sport (VWS)                          2630
Buitenlandse Zaken (BUZA)                                        1796
Verkeer en Waterstaat (VW)                                       1441
Justitie                                                         1333
Sociale Zaken en Werkgelegenheid (SZW)                           1231
Onderwijs, Cultuur en Wetenschappen (OCW)                        1187
Volkshuisvesting, Ruimtelijke Ordening en Milieubeheer (VROM)     984
FinanciÃ«n (FIN)                                                   960
Volksgezondheid, Welzijn en Sport                                 951
Economische Zaken (EZ)                                            946
Buitenlandse Zaken                                                753
Binnenlandse Zaken en Koninkrijksrelaties (BZK)                   725
Verkeer en Waterstaat                                             724
Defensie (DEF)                                                    646
Sociale Zaken en Werkgelegenheid                                  607
Landbouw, Natuurbeheer en Visserij (LNV)                          586
Volkshuisvesting, Ruimtelijke Ordening en Milieubeheer            554
Onderwijs, Cultuur en Wetenschappen                               532
Vreemdelingenzaken en Integratie (VI)                             466
    </pre>
</p>


<h2>Form of handing in your final product</h2>

* An IPython notebook with for each question, a MarkDown cell containing the question, a code cell which solves the question, an output cell with the output, followed by a MarkDown cell with explanation/reflection  

In [1]:
import pandas as pd

names=['jaar', 'partij','titel','vraag','antwoord','ministerie']

# Change to KVR1000.csv.gz if this becomes too slow for you
# kvrdf= pd.read_csv('http://maartenmarx.nl/teaching/zoekmachines/LectureNotes/MySQL/KVR.csv.gz', 
kvrdf= pd.read_csv('http://maartenmarx.nl/teaching/zoekmachines/LectureNotes/MySQL/KVR1000.csv.gz', 
                   compression='gzip', sep='\t', 
                   index_col=0, names=names,
                   ) 

for kolom in names[1:]:
    kvrdf[kolom]= kvrdf[kolom].astype(str)
print(kvrdf.shape)
kvrdf.head()



(1000, 6)


,jaar,partij,titel,vraag,antwoord,ministerie
KVR1000.xml,1994,PvdA,De vragen betreffen de betrouwbaarheid van de...,Hebt u kennisgenomen van het televisieprogram...,Ja. Het bedoelde geluidmeetpunt is eigendom v...,Verkeer en Waterstaat
KVR10000.xml,1999,PvdA,Vragen naar aanleiding van berichten (uitzend...,Kent u de berichten over de situatie in de Me...,,Justitie
KVR10001.xml,1999,SP,"Vragen naar aanleiding van de berichten ""Nede...",Kent u de berichten «Nederland steunt de Soeh...,,Financien
KVR10002.xml,1999,PvdA,Vragen over de gebrekkige opvang van verpleeg...,Kent u het bericht over onderzoek van Nu91 me...,Ja. Het onderzoek van NU’91 wijst uit dat het...,"Volksgezondheid, Welzijn en Sport"
KVR10003.xml,1999,PvdA,Vragen over onbetrouwbaarheid van filemeldingen.,Hebt u kennisgenomen van de berichten over de...,Ja. Nee. Door de waarnemers van het Algemeen ...,Verkeer en Waterstaat


In [2]:
from nltk.corpus import stopwords
import nltk
from collections import Counter
import itertools
from math import log

DutchStop= stopwords.words('dutch')
allvragen= '\n'.join(list(kvrdf.titel))
classes = list(set(list(kvrdf.ministerie)))

Words are treated as lowercase & stopwords are filtered out below

In [3]:
# Term frequencies per word

def strip_string(string):
    """
    
    :param string: string of unfiltered word/symbols
    :return: list of all tokens extracted from the string (lowercasing, stopwords, alpha)
    """
    return [w for w in nltk.word_tokenize(string.lower()) if w.isalpha() and not w in set(DutchStop)]
    
def str_to_tf(string):
    """
    
    :param string: string of unfiltered word/symbols
    :return: dictionary of all term frequencies: occurance of term in string
    """
    return Counter(strip_string(string))

In [4]:
def str_to_tk(string):
    """ returns token count in a string
    """
    return len(strip_string(string))

## 1. 
Normalize the values for "ministerie" and choose 10 ministeries to work with.

In [18]:
def determine_classes(classes):
    """ Remove any class that spans multiple ministeries
    
    :param classes: All classes that occur in kamervragen
    :return:
        norm_classes, a subset of classes with only the classes that span a single ministerie.
    """
    norm_classes = set()
    
    for c in classes:
        add = True

        if c == 'nan':
            continue

        for nc in norm_classes:
            if nc in c:
                add = False
                break
            elif c in nc:
                norm_classes.remove(nc)
                break

        if add:
            norm_classes.add(c)
            
    return norm_classes


def normalize_class(c):
    """ Normalize class c by replacing different representations of the e by a normal e
    
    :param c: str, a single class name
    :return:
        str, a single class name with 'normal' e's and no trailing whitespace.
    """
    nc = ""
    parenthesis = False
    
    for char in c:
        if char == '(':
            parenthesis = True
            
        elif char == 'ë':
            char = 'e'
        elif char == 'Ã':
            char = 'e'
        elif char == '«':
            char = ''
            
        if not parenthesis:
            nc += char
            
        if char == ')':
            parenthesis = False
    
    return nc.strip()


def choose_10_classes(class_rows, norm_classes):
    """ Only return the 10 most occuring classes.
    
    :param class_rows: dict, a dictionary mapping a class name to all rows that class occurs in.
    :param norm_classes: set, all normalized class names of classes that only span a single ministerie.
    
    :return:
        set, the normalized names of the 10 most occuring classes
    """
    count = Counter()
    for key, rows in class_rows.items():
        key = normalize_class(key)
        
        if key in norm_classes:
            count[key] += len(rows)
            
    return set([name for name, _ in count.most_common(10)])


def kvrdf_to_10_classes(kvrdf, norm_classes):
    """ Return a copy of kvrdf with normalized class names and only containing rows with classes in norm_classes
    
    :param kvrdf: pd.Dataframe, containing all downloaded information: questions, ministerie, answers, etc.
    :param norm_classes: set, containing all normalized class names of the 10 most occuring classes.
    
    :return:
        pd.Dataframe, kvrdf but with all rows removed that don't contain any of the ministeries in norm_classes.
    """   
    for c in set(kvrdf.ministerie):
        nc = normalize_class(c)
        
        if not (nc in norm_classes):
            kvrdf = kvrdf[kvrdf.ministerie != c]
            
        else:
            kvrdf.loc[kvrdf["ministerie"] == c, "ministerie"] = nc
            
    return kvrdf

# classes is a set of strings of all classes that are in kvrdf
classes = set(kvrdf.ministerie)

# norm classes is a set of strings of classes that only span a single ministerie with normalized names.
norm_classes = {normalize_class(c) for c in determine_classes(classes)}

# class_rows_full is a dictionary mapping a class name to all rows that contain that class.
class_rows_full = {c:kvrdf.loc[kvrdf.ministerie == c] for c in classes}

# classes is adjusted so it now is a set that contains the normalized strings of the 10 most occuring classes.
classes = choose_10_classes(class_rows_full, norm_classes)
print(classes)

# kvrdf is adjusted so any rows that don't contain any class in 'classes' are removed.
# also, all class names are normalized.
kvrdf = kvrdf_to_10_classes(kvrdf, classes)
kvrdf.head()

{'Defensie', 'Sociale Zaken en Werkgelegenheid', 'Volkshuisvesting, Ruimtelijke Ordening en Milieubeheer', 'Buitenlandse Zaken', 'Justitie', 'Verkeer en Waterstaat', 'Landbouw, Natuurbeheer en Visserij', 'Onderwijs, Cultuur en Wetenschappen', 'Economische Zaken', 'Volksgezondheid, Welzijn en Sport'}


,jaar,partij,titel,vraag,antwoord,ministerie
KVR1000.xml,1994,PvdA,De vragen betreffen de betrouwbaarheid van de...,Hebt u kennisgenomen van het televisieprogram...,Ja. Het bedoelde geluidmeetpunt is eigendom v...,Verkeer en Waterstaat
KVR10000.xml,1999,PvdA,Vragen naar aanleiding van berichten (uitzend...,Kent u de berichten over de situatie in de Me...,,Justitie
KVR10002.xml,1999,PvdA,Vragen over de gebrekkige opvang van verpleeg...,Kent u het bericht over onderzoek van Nu91 me...,Ja. Het onderzoek van NU’91 wijst uit dat het...,"Volksgezondheid, Welzijn en Sport"
KVR10003.xml,1999,PvdA,Vragen over onbetrouwbaarheid van filemeldingen.,Hebt u kennisgenomen van de berichten over de...,Ja. Nee. Door de waarnemers van het Algemeen ...,Verkeer en Waterstaat
KVR10004.xml,1999,D66,Vragen naar aanleiding van het bericht in de ...,Kent u het bericht als zou het RIVM onjuiste ...,Ja. Nee. Als reactie op het Volkskrant-artike...,Verkeer en Waterstaat


## 2.
Implement the two algorithms in Fig MRS.13.2, using your earlier code for creating term and document frequencies. It might be easier to use the representation and formula given in MRS section 13.4.1.

In [6]:

#document frequencies per class per term
def TrainMultinomialNB(pd_df, classes):
    """ more reference from p258 onwards of MRS
    
    :param classes: classes which are taken into account for training
    :param pd_df: kamervragen dataframe
    :return:
        V: set, of terms which which are used for classification
        class_priors: dictionary, prior probabilities for each class: P(c) prior[class]
        cond_prob: dictionary, conditional probabilities for each term per class: P(t|c) cond[class][term]
        dfdict: dictionary with document frequencies
        classes: list, classes actually used, temporary until class input is synchronised with pandas dataframe
    """
    class_frequency = Counter([c for c in pd_df.ministerie])
    
    # classes = classes # uncomment this and comment next line when classes are implemented right 
    classes = class_frequency.keys()
    
    # P(c)
    class_priors = {c:class_frequency[c]/sum([class_frequency[cn] for cn in classes]) for c in classes}

    # document frequencies per term
    k = [list(set(strip_string(t))) for t in kvrdf.titel]
    dfdict = Counter(list(itertools.chain.from_iterable(k)))

    #vocabulary
    V = set(dfdict.keys())
    
    # complete rows per class
    class_rows_full = {c:kvrdf.loc[kvrdf.ministerie == c] for c in classes}

    # term frequency per class
    class_tf = {c:str_to_tf('\n'.join(list(class_rows_full.get(c).titel))) for c in classes}
    
    # token count per class (for sum over Tct' in (t' in V), (p259, formula 13.6)
    class_tk = {c:str_to_tk('\n'.join(list(class_rows_full.get(c).titel))) for c in classes}

    #P(t|c)
    cond_prob = {t:{c:(class_tf[c][t] + 1)/(class_tk[c] + 1) for c in classes} for t in V}
    return V, class_priors, cond_prob, dfdict, classes

def filter_vocab(W, V):
    """ 
    
    :param W: list; stripped query
    :param V: set; of vocab used for classification (determined by one or anoter method)
    :return: a list of words that occur in the used vocabulary
    """
    return [w for w in W if w in V]

def ApplyMultinomialNB(classes, priors, conditionals, d, Vocab=None):
    """ Applies a Naive Bayes classifier on previously derived priors & conditionals on a query d

    :param classes: list, optional ministeries to classify between 
    :param Vocab: set, of terms which which are used for classification
    :param priors: dictionary, prior probabilities for each class: P(c) prior[class]
    :param conditionals: dictionary, conditional probabilities for each term per class: P(t|c) cond[class][term]
    :param d: string, query
    :return: best classification for query
    """
    W = strip_string(d)
    if Vocab:
        W = filter_vocab(W, Vocab)
    score = {c:log(priors[c]) for c in classes}
    for k in score.keys():
        score[k] += sum([log(conditionals[t][k]) for t in W])    
    best_class = sorted(score, key = lambda key: score[key])[0]
    print(sorted(score, key = lambda key: score[key]))
    return best_class



## 3.
On this collection, train NB text classifiers for 10 different classes with enough and interesting data.

In [7]:
V, class_priors, cond_prob, dfdict, classes = TrainMultinomialNB(kvrdf, classes)
ApplyMultinomialNB(classes, class_priors, cond_prob, "vragen")


['Economische Zaken', 'Defensie', 'Landbouw, Natuurbeheer en Visserij', 'Volkshuisvesting, Ruimtelijke Ordening en Milieubeheer', 'Onderwijs, Cultuur en Wetenschappen', 'Sociale Zaken en Werkgelegenheid', 'Buitenlandse Zaken', 'Verkeer en Waterstaat', 'Volksgezondheid, Welzijn en Sport', 'Justitie']


'Economische Zaken'

## 4.
Compute for each term and each of your 10 classes its utility for that class using mutual information.

In [8]:
def extract_class_df(pddf, classes):
    class_dfdict = {c:{} for c in classes}

    for i, c in enumerate(classes):
        for d in kvrdf[kvrdf.ministerie == c].titel:
            for elem in set(strip_string(d)):
                if class_dfdict[c].get(elem, 0):
                    class_dfdict[c][elem] += 1
                else:
                    class_dfdict[c][elem] = 1
    return class_dfdict

cddf = extract_class_df(kvrdf, classes)

In [9]:
def MI(doc_freq, class_doc_freq):
    """ creates a mutual information dictionary for each term, class pair
    
    :param doc_freq: dictionary, contains documents frequencies per term, doc_freq[term]
    :param class_doc_freq: dictionary, contains documents frequencies per term per class, cdf[class][term]
    :return MIdict: dictionary, contains mutual information for each class/term pair, MI[class][term]
    """
    N = sum(doc_freq.values())
    MIdict = {}
    for c in class_doc_freq.keys():
        MIdict[c] = {}
        for t in class_doc_freq[c].keys():
            N11 = class_doc_freq[c][t]
            N10 = doc_freq[t] - N11
            N01 = sum(class_doc_freq[c].values()) - class_doc_freq[c][t]
            N00 = N - doc_freq[t] - (sum(class_doc_freq[c].values()) - class_doc_freq[c][t])
            N1 = N11 + N10
            N0 = N01 + N00
            if N11:
                MIdict[c][t] = N11/N * log( (N*N11)/(N1*N1), 2)
            if N01:
                MIdict[c][t] += N01/N * log( (N*N01)/(N0*N1) , 2)
            if N10:
                MIdict[c][t] += N10/N * log( (N*N10)/(N1*N0) , 2)
            if N00:
                MIdict[c][t] += N00/N * log(  (N*N00)/(N0*N0), 2)
    return MIdict

MIdict = MI(dfdict, cddf)

## 5.
For each class, show the top 10 words as in Figure 13.7 in MRS.

In [10]:
from IPython.display import display, Markdown, Latex

display(Markdown(
"""
|some| markdown|
|-----|------|
| test1 | test2 | 
| test3 | test4 |
"""))



|some| markdown|
|-----|------|
| test1 | test2 | 
| test3 | test4 |


In [11]:
def top_MI(MIdict, N):
    """ 
    
    :param MIdict: a dictionary with mutual information for each MIdict[class][term]
    :param N: top N MI terms returned per class
    :return: a dictionary of the top N MI predictors of each class    
    """
    resdict = {}
    for c in MIdict.keys():
        resdict[c] = sorted(MIdict[c].items(), key = lambda key: -MIdict[c][key[0]])[:N]
    return resdict

def top_MI_to_markdown(topMI):
    firstline = classes_table_markdown(topMI)
    rest_of_table = values_table_markdown(topMI)
    return firstline + rest_of_table


def classes_table_markdown(topmidict):
    return "|" + " | ".join(topmidict.keys()) + "|\n" + "|" + " | ".join(['-----' for _ in topmidict.keys()]) + "|"

def values_table_markdown(topmidict):
    returnstring = ""

    for i, elem in enumerate(topmidict[list(topmidict.keys())[0]]):
        returnstring += " | " + " | ".join([pair_to_markdown(topmidict[c][i]) for c in topmidict.keys()]) + " | \n"
    return returnstring
            
def pair_to_markdown(pair):
    return pair[0] + " | " + str(pair[1])


def topMI_to_vocab(topmidict):
    vocab_l_o_l = [[elem[0] for elem in topmidict[c]] for c in topmidict.keys()]
    return set(list(itertools.chain.from_iterable(vocab_l_o_l)))



In [12]:
topfeatures = top_MI(MIdict, 1)
for c in topfeatures.keys():
    print(len(topfeatures[c]))



1
1
1
1
1
1
1
1
1
1


In [13]:
topfeatures_MD = top_MI_to_markdown(topfeatures)
# bug is because of empty classes & not being able to recognize classes in normalized form

display(Markdown(topfeatures))

TypeError: Markdown expects text, not {'Verkeer en Waterstaat': [('geluidsmetingen', 0.9064642091914905)], 'Justitie': [('twijfels', 1.930452001280508)], 'Volksgezondheid, Welzijn en Sport': [('traumatische', 1.5588902321464466)], 'Defensie': [('bodem', 0.30255954304494675)], 'Landbouw, Natuurbeheer en Visserij': [('envirodung', 0.3896423179577661)], 'Buitenlandse Zaken': [('veiligheidsraad', 1.0017786861976063)], 'Economische Zaken': [('clyde', 0.4346903879604562)], 'Volkshuisvesting, Ruimtelijke Ordening en Milieubeheer': [('voetnoten', 0.7415464892478193)], 'Sociale Zaken en Werkgelegenheid': [('werkplek', 0.9046081268620453)], 'Onderwijs, Cultuur en Wetenschappen': [('onvrijwillige', 0.7542116195739694)]}

## 6.
Evaluate your classifiers using Precision, Recall and F1. (Give a table in which you show these values for using the top 10, top 100 terms and all terms, for all of your 10 classes)
          Thus do feature selection per class, and use for each class the top n best features for that class. 
          <br/>
      Also show the microaverage(s) for all 10 classes together.
      <br/>
      If you like you can also present this in a figure like MRS.13.8. 
      Then compute the F1 measure for the same number of terms as in that figure.

In [ ]:


def precision():
    raise NotImplemented
    
def recall():
    raise NotImplemented
    
def F1_measure():
    P = precision()
    R = recall()
    return 2*P*R/(P+R)

In [ ]:
def getstats()
    ApplyMultinomialNB(classes, priors, conditionals, d, Vocab=None)

## 7 
You have done the complete implementation by yourself. Congratulations! You can also use `scikit-learn` routines for all of this work. Do that. So follow [this text classification tutorial](http://scikit-learn.org/stable/tutorial/text_analytics/working_with_text_data.html)  and implement the same steps but now with your kamervragen dataset. Also use [mutual information feature selection](http://scikit-learn.org/stable/modules/feature_selection.html) to select the K-best features, and compare the results as before.


## 8

Reflect and report briefly about your choices in this process and about the obtained results. Also reflect on the differences between the scikit learn approach and the "own implementation approach".

<h3>Training/Testing</h3>
<p>It is important that you do not test your classifier using documents that have also been used in training.
    So split up your collection in a training set and a test set. A 80%-20% split is reasonable.

<br/>
    If you have too little data you can use 5 or <a href="http://en.wikipedia.org/wiki/Cross-validation_(statistics)#k-fold_cross-validation">10-fold cross validation</a>.</p>



<h2>Form of handing in your final product</h2>

* An IPython notebook with for each question, a MarkDown cell containing the question, a code cell which solves the question, an output cell with the output, followed by a MarkDown cell with explanation/reflection  